In [1]:
##WIND Vs Windy Tweets

In [2]:
import scipy
import numpy as np
import pandas as pd
import plotly.plotly as py
# import visplots
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from sklearn import preprocessing, metrics
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from scipy.stats.distributions import randint
import plotly.graph_objs as go
import plotly.tools as tls
tls.set_credentials_file(username='cap7carps', api_key='d28TqBXnTSA80L2xctmI')
from numpy import arange,array,ones
from scipy import stats

In [52]:
weatherFrame = pd.read_pickle(r'C:\GIT\weatherproj\Weather\weather_data.pki')

In [53]:
with open (r'C:\GIT\weatherproj\Weather\glastweetsy.json', 'rb') as glas:
    gdata = glas.readlines()

gdata = map(lambda x: x.rstrip(), gdata)

gdatajson = "[" + ','.join(gdata) + "]"

glasgow = pd.read_json(gdatajson)

In [155]:
with open (r'C:\GIT\weatherproj\Weather\london_tweets.json', 'rb') as lon:
    data = lon.readlines()

data = map(lambda x: x.rstrip(), data)

datajson = "[" + ','.join(data) + "]"

london = pd.read_json(datajson)

In [55]:
glasgow.head()

,Cold,Rain,W1,W2,Warm,Windy,_id,city,coordinates,created_at,date_time_period,text
0,Y,N,Y,Y,N,N,{u'$oid': u'586b64f713dc7218c8ac3ba9'},Glasgow,None,{u'$date': u'2016-12-08T14:49:54.000Z'},{u'$date': u'2016-12-08T12:00:00.000Z'},Nice graphic though hey?
1,Y,N,Y,Y,N,N,{u'$oid': u'586b64f713dc7218c8ac3bb9'},Glasgow,None,{u'$date': u'2016-12-08T14:53:42.000Z'},{u'$date': u'2016-12-08T12:00:00.000Z'},We're counting down the days till Christmas he...
2,Y,N,Y,Y,N,N,{u'$oid': u'586b64f713dc7218c8ac3c12'},Glasgow,"{u'type': u'Point', u'coordinates': [-3.1603, ...",{u'$date': u'2016-12-08T15:02:22.000Z'},{u'$date': u'2016-12-08T15:00:00.000Z'},It's Kate Moss bitches! 🙌❤️\n.\n#markvice #ma...
3,N,N,Y,Y,N,Y,{u'$oid': u'586b64f713dc7218c8ac3c44'},Glasgow,None,{u'$date': u'2016-12-08T15:07:26.000Z'},{u'$date': u'2016-12-08T15:00:00.000Z'},"""aspens shiver in the wind - transcend-ent, i..."
4,N,N,Y,Y,N,Y,{u'$oid': u'586b64f713dc7218c8ac3c80'},Glasgow,"{u'type': u'Point', u'coordinates': [-4.25, 55...",{u'$date': u'2016-12-08T15:14:32.000Z'},{u'$date': u'2016-12-08T15:00:00.000Z'},overcast clouds -&gt; broken clouds\nhumidity ...


In [101]:
weatherFrame.groupby('city').mean()

,avg_rain_prob,avg_temp_frc,forecast_period,avg_temp_obs,avg_ws_obs,avg_temp_dif,rain_accuracy
city,,,,,,,
glasgow,27.126554,6.399011,51.194350,6.705532,10.258969,1.177688,0.710703
london,13.553618,6.997349,51.294493,6.935751,7.370902,1.205872,0.789105


In [123]:
windvals = weatherFrame.ix[weatherFrame['city']=='glasgow',['date_time_period', 'avg_ws_obs']].groupby('date_time_period').mean()

In [124]:
windvals.head()

,avg_ws_obs
date_time_period,
2016-12-08 09:00:00,4.666667
2016-12-08 12:00:00,9.666667
2016-12-08 15:00:00,7.333333
2016-12-08 18:00:00,5.000000
2016-12-08 21:00:00,2.666667


In [125]:
type(london.ix[0,9])

dict

In [126]:
type(weatherFrame.ix[0,3])

pandas.tslib.Timestamp

In [127]:
glasgow2 = pd.read_pickle(r'C:\GIT\weatherproj\Weather\glaswindpickle')

In [128]:
glasgow2.head()

,Cold,Rain,W1,W2,Warm,Windy,_id,city,coordinates,created_at,date_time_period,text
0,N,Y,Y,Y,N,Y,586ce27c13dc7211f4de70ca,Glasgow,"{u'type': u'Point', u'coordinates': [-3.143, 5...",2016-12-08 15:00:47,2016-12-08 15:00:00,"15:00 GMT: Temperature: -, Wind: , Humidity: ,..."
1,N,N,Y,Y,N,Y,586ce27c13dc7211f4de710b,Glasgow,None,2016-12-08 15:07:26,2016-12-08 15:00:00,"""aspens shiver in the wind - transcend-ent, i..."
2,N,N,Y,Y,N,Y,586ce27c13dc7211f4de725c,Glasgow,"{u'type': u'Point', u'coordinates': [-4.1, 55....",2016-12-08 15:47:25,2016-12-08 15:00:00,"current weather in Blantyre: broken clouds, 10..."
3,N,N,Y,Y,N,Y,586ce27c13dc7211f4de7260,Glasgow,"{u'type': u'Point', u'coordinates': [-4.25, 55...",2016-12-08 15:47:51,2016-12-08 15:00:00,"current weather in Glasgow: broken clouds, 10°..."
4,N,Y,Y,Y,N,Y,586ce27c13dc7211f4de72fb,Glasgow,"{u'type': u'Point', u'coordinates': [-3.143, 5...",2016-12-08 16:03:11,2016-12-08 15:00:00,"16:00 GMT: Temperature: -, Wind: , Humidity: ,..."


In [129]:
gwindcount = glasgow2.ix[:,['date_time_period','text']].groupby('date_time_period').agg('count')
gwindcount.head()

,text
date_time_period,
2016-12-08 15:00:00,6
2016-12-08 18:00:00,4
2016-12-08 21:00:00,3
2016-12-09 00:00:00,3
2016-12-09 03:00:00,3


In [168]:
london2 = pd.read_pickle(r'C:\GIT\weatherproj\Weather\lonwindpickle.pki')
lwindcount = london2.ix[:,['date_time_period','text']].groupby('date_time_period').agg('count')
lwindcount.head()

,text
date_time_period,
2016-12-07 18:00:00,7
2016-12-07 21:00:00,10
2016-12-08 00:00:00,6
2016-12-08 03:00:00,8
2016-12-08 06:00:00,12


In [158]:
london2.head()

,Cold,Rain,W1,W2,Warm,Windy,_id,city,coordinates,created_at,date_time_period,text
0,N,Y,Y,Y,N,Y,586b64e18d709f1f100a0979,london,"{u'type': u'Point', u'coordinates': [-0.237222...",2016-12-07 18:59:27,2016-12-07 18:00:00,"Changeable, some precipitation. Now:12.1C - Ra..."
1,N,Y,Y,Y,N,Y,586b64e18d709f1f100a09d2,london,"{u'type': u'Point', u'coordinates': [-0.469791...",2016-12-07 19:00:42,2016-12-07 18:00:00,"18:52 Temp. 11.1°C, Hum. 85%, Dewp. 8°C, Bar. ..."
2,N,N,Y,Y,N,Y,586b64e28d709f1f100a1510,london,None,2016-12-07 19:40:01,2016-12-07 18:00:00,just to personally wind you up Kano?
3,N,Y,Y,Y,N,Y,586b64e38d709f1f100a1a87,london,"{u'type': u'Point', u'coordinates': [-0.237222...",2016-12-07 19:59:29,2016-12-07 18:00:00,"Changeable, some precipitation. Now:12.3C - Ra..."
4,N,Y,Y,Y,N,Y,586b64e38d709f1f100a1af9,london,"{u'type': u'Point', u'coordinates': [-0.469791...",2016-12-07 20:00:44,2016-12-07 18:00:00,"19:53 Temp. 10.9°C, Hum. 85%, Dewp. 7.8°C, Bar..."


In [133]:
gvals = gwindcount.text.values

In [134]:
dates = gwindcount.index

In [136]:
glaswindcount = windvals.join(gwindcount)

In [137]:
glaswindcount.head()

,avg_ws_obs,text
date_time_period,,
2016-12-08 09:00:00,4.666667,NaN
2016-12-08 12:00:00,9.666667,NaN
2016-12-08 15:00:00,7.333333,6.0
2016-12-08 18:00:00,5.000000,4.0
2016-12-08 21:00:00,2.666667,3.0


In [138]:
glaswindcount.text.values

array([ nan,  nan,   6.,   4.,   3.,   3.,   3.,   5.,   5.,   6.,   5.,
         6.,   4.,   3.,   1.,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,   4.,  nan,   1.,  nan,  nan,   2.,  nan,
         2.,   4.,  nan,  nan,  nan,  nan,   3.,  nan,   3.,   3.,   2.,
         1.,  nan,  nan,  nan,   1.,   4.,   2.,  nan,  nan,  nan,  nan,
         2.,  nan,   5.,   2.,  nan,   2.,  nan,  nan,   2.,   1.,   3.,
         2.,  nan,   1.,  nan,  nan,   2.,  nan,   2.,   5.,  nan,  nan,
         1.,   1.,   2.,   1.,   2.,   2.,  nan,  nan,  nan,  nan,   2.,
         2.,   2.,   3.,   2.,  nan,  nan,  nan,   4.,   5.,   4.,   6.,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,   4.,
         9.,  15.,   7.,   2.,   7.,   2.,   7.,   6.,  12.,   5.,   2.,
         5.,  nan,  nan,   2.,   4.,   5.,   3.,   2.,   4.,   3.,   1.,
         3.,   1.,   8.,  11.,   1.,   1.,  nan,  n

In [184]:
line1 = go.Scatter(
    x = list(glaswindcount.index),
    y = list(glaswindcount.avg_ws_obs.values),
    mode = 'lines',
    name = 'wind speed'
)


line2 = go.Scatter(
    x = list(glaswindcount.index),
    y = list(glaswindcount.text.values),
    mode = 'lines',
    name='Count of Wind Related Tweets',
    yaxis='y2'
)

layout = go.Layout(
    title='Count vs Speed',
    
    yaxis=dict(
        title='Speed',
        titlefont=dict(
            color='rgb(0, 100, 200)'))
    
,    
    xaxis=dict(title='Date')
,
    yaxis2=dict(
        title='Count of Tweets',
        titlefont=dict(
            color='rgb(255,128,0)'
        ),        
        overlaying='y',
        side='right'
    ))

data = [line1, line2]
fig = go.Figure(data=data, layout=layout)
# Plot and embed in ipython notebook!
py.iplot(fig)

In [153]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Scatter(
    x = list(glaswindcount.index),
    y = list(glaswindcount.avg_ws_obs.values),
    name='Wind Speed'
)
trace2 = go.Scatter(
    x = list(glaswindcount.index),
    y = list(glaswindcount.text.values),
    name='Count of Weather Related Tweets',
    yaxis='y2'
)
data = [trace1, trace2]
layout = go.Layout(
    title='Glasgow - Wind Speed vs Wind Related Tweets',
    yaxis=dict(
        title='Wind Speed'
    ),
    yaxis2=dict(
        title='Count of Tweets',
        titlefont=dict(
            color='rgb(0, 100, 200)'
        ),        
        overlaying='y',
        side='right'
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [161]:
lwindcount.head()

,text
date_time_period,
2016-12-08 15:00:00,6
2016-12-08 18:00:00,4
2016-12-08 21:00:00,3
2016-12-09 00:00:00,3
2016-12-09 03:00:00,3


In [162]:
lonwindvals = weatherFrame.ix[weatherFrame['city']=='london',['date_time_period', 'avg_ws_obs']].groupby('date_time_period').mean()

In [163]:
lonwindvals.head()

,avg_ws_obs
date_time_period,
2016-12-08 09:00:00,13.000000
2016-12-08 12:00:00,12.888889
2016-12-08 15:00:00,9.111111
2016-12-08 18:00:00,6.777778
2016-12-08 21:00:00,6.777778


In [169]:
londonboth = lwindcount.join(lonwindvals)

In [170]:
londonboth.head()

,text,avg_ws_obs
date_time_period,,
2016-12-07 18:00:00,7,NaN
2016-12-07 21:00:00,10,NaN
2016-12-08 00:00:00,6,NaN
2016-12-08 03:00:00,8,NaN
2016-12-08 06:00:00,12,NaN


In [177]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Scatter(
    x = list(londonboth.index),
    y = list(londonboth.avg_ws_obs.values),
    name='Wind Speed'
)
trace2 = go.Scatter(
    x = list(londonboth.index),
    y = list(londonboth.text.values),
    name='Count of Weather Related Tweets',
    yaxis='y2'
)
data = [trace1, trace2]
layout = go.Layout(
    title='London - Wind Speed vs Wind Related Tweets',
    yaxis=dict(
        title='Wind Speed'
    ),
    yaxis2=dict(
        title='Count of Tweets',
        titlefont=dict(
            color='rgb(0, 100, 200)'
        ),        
        overlaying='y',
        side='right'
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [172]:
london2.head()

,Cold,Rain,W1,W2,Warm,Windy,_id,city,coordinates,created_at,date_time_period,text
0,N,Y,Y,Y,N,Y,586b64e18d709f1f100a0979,london,"{u'type': u'Point', u'coordinates': [-0.237222...",2016-12-07 18:59:27,2016-12-07 18:00:00,"Changeable, some precipitation. Now:12.1C - Ra..."
1,N,Y,Y,Y,N,Y,586b64e18d709f1f100a09d2,london,"{u'type': u'Point', u'coordinates': [-0.469791...",2016-12-07 19:00:42,2016-12-07 18:00:00,"18:52 Temp. 11.1°C, Hum. 85%, Dewp. 8°C, Bar. ..."
2,N,N,Y,Y,N,Y,586b64e28d709f1f100a1510,london,None,2016-12-07 19:40:01,2016-12-07 18:00:00,just to personally wind you up Kano?
3,N,Y,Y,Y,N,Y,586b64e38d709f1f100a1a87,london,"{u'type': u'Point', u'coordinates': [-0.237222...",2016-12-07 19:59:29,2016-12-07 18:00:00,"Changeable, some precipitation. Now:12.3C - Ra..."
4,N,Y,Y,Y,N,Y,586b64e38d709f1f100a1af9,london,"{u'type': u'Point', u'coordinates': [-0.469791...",2016-12-07 20:00:44,2016-12-07 18:00:00,"19:53 Temp. 10.9°C, Hum. 85%, Dewp. 7.8°C, Bar..."


In [176]:
weatherFrame.ix[:,['forecast_period', 'rain_accuracy']].groupby('forecast_period').mean()

,rain_accuracy
forecast_period,
-5.0,0.710455
-4.0,0.767105
-3.0,0.787941
-2.0,0.579333
-1.0,0.779071
0.0,0.825500
1.0,0.593824
2.0,0.781953
3.0,0.778578
